https://medium.com/@AmyGrabNGoInfo/sentiment-analysis-hugging-face-zero-shot-model-vs-flair-pre-trained-model-57047452225d

## Load the data

In [1]:
# !pip install pandas numpy 
import pandas as pd
import numpy as np

In [2]:
# df = pd.read_csv("/content/sample_data/tweet_AAPL.csv")
df = pd.read_csv("./theCleanedTweets.csv")
df = df[['date','tweetV2']]
df = df.reindex(index = df.index[::-1]).reset_index(drop=True)

/tmp/ipykernel_10740/659938449.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./theCleanedTweets.csv")


In [3]:
# !pip install transformers


In [4]:
from transformers import pipeline

/home/bassturtle4/miniconda3/envs/env/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/bassturtle4/miniconda3/envs/env/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [5]:
# Define pipeline
classifier = pipeline(task="zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device='cuda')

In [6]:
# !pip3 uninstall torch torchvision torchaudio -y

In [7]:
# !pip3 install pillow
# !pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/rocm5.7
# # !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/rocm5.6

In [8]:
#
small_df = df.sample(frac=.05, random_state=42)
print(small_df.shape)
print(small_df.head())

(5809, 2)
            date                                            tweetV2
37451   2021/5/3                       do you have pt for aapl gary
38591   2021/5/7  this is amazing a good example of where the ba...
16511  2021/2/18     aapl is still my most profitable trade to date
24840  2021/3/17  too many calls after er beat so they fabricate...
34376  2021/4/28  aapl closed today at if you bought share of aa...


In [9]:
df['tweetV2'].map(type) != str
df = df[df['tweetV2'].apply(lambda x: isinstance(x, str))]

### Neutral

In [10]:
from tqdm import tqdm
# Put reviews in a list
#sequences = small_df.head(20)['tweetV2'].to_list()

sequences = df['tweetV2'].to_list()

candidate_labels = ["positive", "neutral", "negative"]

hypothesis_template = "The sentiment of this review is {}."

predictions = []

for sequence in tqdm(sequences, desc="Classifying Sentiments"):
    result = classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)
    predictions.append(result)

hf_prediction = pd.DataFrame(predictions)

Classifying Sentiments:   0%|                             | 0/116174 [00:00<?, ?it/s]


RuntimeError: HIP error: invalid device function
HIP kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing HIP_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_HIP_DSA` to enable device-side assertions.


In [ ]:
# import concurrent.futures
# from tqdm import tqdm

# sequences = df['tweetV2'].to_list()
# candidate_labels = ["positive", "neutral", "negative"]
# hypothesis_template = "The sentiment of this review is {}."

# def classifySentiment(sequence):
#     return classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

# num_threads = 50

# sequence_chunks = [sequences[i:i + num_threads] for i in range(0, len(sequences), num_threads)]

# predictions = []

# with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
#     for chunk in tqdm(sequence_chunks, desc="Classifying Sentiments"):
#         results = list(executor.map(classifySentiment, chunk))
#         predictions.extend(results)

# hf_prediction = pd.DataFrame(predictions)


In [ ]:
# Take a look at the data
hf_prediction.head(10)

In [ ]:
hf_prediction.iloc[2]

In [ ]:
hf_prediction.iloc[2].sequence

### TO DO: We need to sort the column "labels" and "scores" to have the labels "positive, neutral, and negative" matching the scores.

### If possible, have three columns, "positive", "neutral", and "negative"

In [ ]:
labelOreo = ['positive', 'neutral', 'negative']

def labelSort(row):
    labels, scores = row['labels'].tolist(), row['scores']
    
    label_index = {label: labelOreo.index(label) for label in labels}
    
    sorted_labels = sorted(labels, key=lambda x: label_index[x])
    
    sorted_scores = [scores[labels.index(label)] for label in sorted_labels]
    
    return sorted_labels, sorted_scores

hf_prediction[['labels','scores']] = hf_prediction.apply(labelSort, axis=1, result_type='expand')


In [ ]:
hf_prediction

In [ ]:
hf_prediction.to_csv("./20HoursOfPain.csv", index=False)